### Домашнее задание к уроку 3<a class="anchor" id="hw"></a><center>

In [178]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [179]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [180]:
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 5.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [181]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [182]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [183]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [184]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

#### __1__ *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [185]:
def calc_logloss(y, y_pred):
# Лазарева добавлено (
    corr_coef = 1e-15

    y_pred = np.where(y_pred == 0, 0 + corr_coef, y_pred)
    y_pred = np.where(y_pred == 1, 1 - corr_coef, y_pred)
    
# Лазарева добавлено )
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [186]:
# Пример применения
y1 = np.array([0.1, 0])
y_pred1 = np.array([0, 1])
calc_logloss(y1, y_pred1)

18.996726815915974

In [187]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [188]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
            
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
#             Лазарева (
#            закомментировано
#             print(i, W, err)                    
#             Лазарева )
            
    return W, err

#### __2__ Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [202]:
it = np.array([10,50,100,250,500,750,1000,1250,1500,2500,3000,5000])

In [203]:
et = np.array([1e-4,1e-2,1,2,5,10,12,15,20, 23, 25, 30, 50, 75, 100])

In [204]:
error = 100
iters = 0
etas = 0
for i in tqdm(it):
    for j in et:
        W, err = eval_model(X_st, y, iterations=i, eta=j)
        
        if err < error:
            error = err
            iters = i
            etas = j
   

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-187-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.67it/s]


In [205]:
print('iterations ', iters, ', eta', etas, ', error',  error)

iterations  5000 , eta 100.0 , error 0.0002408170061795746


#### __3__ Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [208]:
def calc_pred_proba(W,X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [209]:
calc_pred_proba(W,X_st)

array([1.20916461e-003, 5.15359384e-027, 1.00000000e+000, 1.36521947e-147,
       1.00000000e+000, 1.23664199e-005, 1.00000000e+000, 8.71024113e-016,
       9.98814916e-001, 1.00000000e+000])

__4__ Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [210]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W,X) 
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [211]:
calc_pred(W,X_st)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])